In [30]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xzf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SpotifyAssignment").getOrCreate()
print("Spark version:", spark.version)


Spark version: 3.3.0


In [5]:
from google.colab import files
uploaded = files.upload()


Saving tracks.csv to tracks.csv


In [6]:
df = spark.read.csv("tracks.csv", header=True, inferSchema=True)


In [7]:
df.printSchema()


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- id_artists: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [8]:
df.show(10)


+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|['45tIt06XoI0Iio4...|  1922-02-22|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


In [9]:
print("Total number of tracks:", df.count())


Total number of tracks: 586672


In this part, I set up Apache Spark in Google Colab, uploaded the `tracks.csv` file from the Spotify dataset, and loaded it into a Spark DataFrame. I displayed the schema and sample data to inspect the structure and confirmed the total number of tracks in the dataset.


In [12]:
from pyspark.sql.functions import year, to_date, col
df = df.withColumn("release_date", to_date(col("release_date"), "yyyy-MM-dd"))
df = df.withColumn("year", year(col("release_date")))


In [13]:
selected_columns = ['id', 'name', 'artists', 'year', 'popularity', 'danceability', 'energy',
                    'loudness', 'tempo', 'valence', 'acousticness', 'instrumentalness']
df_selected = df.select(selected_columns)
df_selected.show(5)


+--------------------+--------------------+-------------------+----+----------+------------+------+--------+-------+-------+------------+----------------+
|                  id|                name|            artists|year|popularity|danceability|energy|loudness|  tempo|valence|acousticness|instrumentalness|
+--------------------+--------------------+-------------------+----+----------+------------+------+--------+-------+-------+------------+----------------+
|35iwgR4jXetI318WE...|               Carve|            ['Uli']|1922|         6|       0.645| 0.445| -13.338|104.851|  0.127|       0.674|           0.744|
|021ht4sdgPcrDgSk7...|Capítulo 2.16 - B...|['Fernando Pessoa']|1922|         0|       0.695| 0.263| -22.136|102.009|  0.655|       0.797|             0.0|
|07A5yehtSnoedViJA...|Vivo para Querert...|['Ignacio Corsini']|1922|         0|       0.434| 0.177|  -21.18|130.418|  0.457|       0.994|          0.0218|
|08FmqUhxtyLTn6pAh...|El Prisionero - R...|['Ignacio Corsini']|1922|  

In [14]:
from pyspark.sql.functions import col
numeric_columns = ['popularity', 'danceability', 'energy', 'loudness',
                   'tempo', 'valence', 'acousticness', 'instrumentalness']
for column_name in numeric_columns:
    df_selected = df_selected.withColumn(column_name, col(column_name).cast("double"))
df_selected.printSchema()


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- popularity: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)



In [15]:
df_selected.describe(['popularity', 'danceability', 'energy', 'loudness', 'tempo', 'valence']).show()


+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|        popularity|      danceability|            energy|          loudness|             tempo|           valence|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|  count|            584818|            584386|            585813|            586404|            586649|            586645|
|   mean| 27.63313714694144|25.703018887344914|18.099465719826494|-8.171628431405718| 117.8753380945041|0.5575767782732828|
| stddev|18.353979388955864|2349.0695544898185| 2148.621270148684|464.58488981021577|30.951486860700278|3.6054750307252235|
|    min|               0.0|               0.0|               0.0|             -60.0|           -31.627|             -25.4|
|    max|             100.0|          463733.0|          617626.0|          176613.0|           246.381|            1951.0|
+-------

In [16]:
from pyspark.sql.functions import col, sum as spark_sum
null_counts = df_selected.select(
    [spark_sum(col(c).isNull().cast("int")).alias(c + "_nulls") for c in ['id', 'name', 'artists', 'year']]
)
null_counts.show()


+--------+----------+-------------+----------+
|id_nulls|name_nulls|artists_nulls|year_nulls|
+--------+----------+-------------+----------+
|       0|        71|            0|    140254|
+--------+----------+-------------+----------+



In [17]:
unique_artists_count = df_selected.select("artists").distinct().count()
print(f"Approximate number of unique artists (string pattern): {unique_artists_count}")


Approximate number of unique artists (string pattern): 113566


The `artists` column is a string representation of a list of artists, for example:  
`['Artist1', 'Artist2']`. For this analysis, I treat it as a single string without parsing it.  
This means artist combinations are counted as unique entries, which may overestimate the number of unique artists.


In [18]:
from pyspark.sql.functions import avg
yearly_trends = df_selected.groupBy("year").agg(
    avg("popularity").alias("avg_popularity"),
    avg("danceability").alias("avg_danceability"),
    avg("energy").alias("avg_energy")
).orderBy("year")
yearly_trends.show(10)


+----+--------------------+------------------+-------------------+
|year|      avg_popularity|  avg_danceability|         avg_energy|
+----+--------------------+------------------+-------------------+
|null|   21.17251445086705|107.01577668517277|  74.25454758294386|
|1900|                19.0|             0.659|              0.791|
|1922|0.043478260869565216|0.5461159420289855|0.21864413043478256|
|1923|0.002481389578163...|0.6623027295285364|0.23874863523573184|
|1924| 0.00684931506849315|0.5117945205479454| 0.3185239726027399|
|1925|0.006309148264984227| 0.619179810725552|0.17728706624605667|
|1926|  0.3780160857908847|0.5446461126005367| 0.2991974530831098|
|1927| 0.06398996235884567|0.6890890840652446|0.24195766624843135|
|1928|  0.0518018018018018|0.6336463963963961|0.25954459459459456|
|1929| 0.05045871559633028|0.6262155963302753|0.25745573394495436|
+----+--------------------+------------------+-------------------+
only showing top 10 rows



In [19]:
yearly_trends.orderBy(yearly_trends.year.desc()).show(10)


+----+------------------+------------------+------------------+
|year|    avg_popularity|  avg_danceability|        avg_energy|
+----+------------------+------------------+------------------+
|2021| 35.17200701418779|0.6714221425155427|0.6180626515223987|
|2020| 44.68210586881473|0.6571431602416566| 0.639573194663405|
|2019|44.909405022754086|0.6496805073318723| 0.636978109683129|
|2018| 42.14782768777614|0.6344810198821795|0.6526337316918269|
|2017| 42.22720326031585|0.6249343759551712| 0.659636893591444|
|2016| 39.33186409550046| 0.603657134986226|0.6598237548668505|
|2015| 37.68623223909848|0.5949681626653602|0.6463989352768235|
|2014|37.055783965613905|0.5985827415436381|0.6702201844234731|
|2013|  36.3056400892676|0.5904964495840954|0.6788864931020484|
|2012| 36.92669463457717|0.5909922609847643|0.6763380326672558|
+----+------------------+------------------+------------------+
only showing top 10 rows




Analyzing the last 10 years (2012–2021), we observe the following:

- **Popularity:** There is a general upward trend in average popularity, increasing from about 37 in 2012 to nearly 45 in 2019, with a slight dip in 2020 and 2021. This suggests that recent tracks tend to have higher popularity scores.

- **Danceability:** Danceability shows a gradual increase from around 0.59 in 2012 to approximately 0.67 in 2021, indicating that newer tracks are somewhat more danceable.

- **Energy:** The average energy remains fairly stable over the years, hovering around 0.63 to 0.67, suggesting consistent energetic qualities in popular music.

Early 1900s data shows much lower popularity and energy, likely reflecting limited or sparse recordings from that era.


In [20]:
top_danceable = df_selected.orderBy(df_selected.danceability.desc()).select("name", "artists", "year", "danceability").limit(10)
top_danceable.show(truncate=False)


+-----------------------------------------------------------------------------------------------------------+-------------------+----+------------+
|name                                                                                                       |artists            |year|danceability|
+-----------------------------------------------------------------------------------------------------------+-------------------+----+------------+
|"From ""Serenade"" K. 185 (167a)                                                                           | Contrabass        |null|463733.0    |
|"Wagner: Die Meistersinger von Nürnberg, WWV 96, Act 3: ""Morgenlich leuchtend im rosigen Schein"" (Walther| Pogner            |null|455240.0    |
|"Puccini: Madama Butterfly, Act 1: ""Ed eccoci in famigila"" ( Butterfly                                   | Bonzo             |null|333293.0    |
|"Bellini: I Puritani, Act 3: ""Credeasi                                                                    | Ri

In [21]:
top_energetic = df_selected.orderBy(df_selected.energy.desc()).select("name", "artists", "year", "energy").limit(10)
top_energetic.show(truncate=False)


+-------------------------------------------------------------------------------------------------------------------+----------+----+--------+
|name                                                                                                               |artists   |year|energy  |
+-------------------------------------------------------------------------------------------------------------------+----------+----+--------+
|"Verdi : Simon Boccanegra : Act 1 ""Messeri                                                                        | Chorus   |null|617626.0|
|"Wagner : Lohengrin : Act 1 ""Nun höret mich und achtet wohl"" [Heerrufer                                          | König    |null|429267.0|
|"Wagner : Lohengrin : Act 1 ""Dank                                                                                 | Chorus   |null|406000.0|
|"Bellini: I Puritani, Act 1: ""A te                                                                                | Giorgio  |null|389427.0|


The top 10 most danceable and energetic tracks in the dataset predominantly feature classical and opera pieces, many of which have exceptionally high numeric values for danceability and energy. These unusually large values suggest possible data inconsistencies or different measurement scales for such genres.

While these tracks showcase extreme values, it highlights how diverse the dataset is, including genres with very different audio characteristics than typical popular music.

In general, one would expect popular dance tracks to have high danceability scores and energetic tracks to have high energy scores, but the presence of classical pieces here suggests a need to consider data cleaning or genre filtering for targeted analyses.


In [22]:
top_artists = df_selected.groupBy("artists").count().orderBy("count", ascending=False).limit(20)
top_artists.show(truncate=False)


+------------------------------------------+-----+
|artists                                   |count|
+------------------------------------------+-----+
|['Die drei ???']                          |3856 |
|['TKKG Retro-Archiv']                     |2006 |
|['Benjamin Blümchen']                     |1503 |
|['Bibi Blocksberg']                       |1472 |
|['Lata Mangeshkar']                       |1373 |
|0                                         |1089 |
|['Bibi und Tina']                         |927  |
|['Tintin', 'Tomas Bolme', 'Bert-Åke Varg']|905  |
|['Francisco Canaro']                      |891  |
|['Ella Fitzgerald']                       |869  |
|['Tadeusz Dolega Mostowicz']              |838  |
|['Fünf Freunde']                          |812  |
|['Mohammed Rafi']                         |787  |
|['Queen']                                 |777  |
|['Elvis Presley']                         |680  |
|['Frank Sinatra']                         |680  |
|['The Rolling Stones']        

In [23]:
top_artist_names = [row['artists'] for row in top_artists.collect()]
from pyspark.sql.functions import col
avg_popularity_top_artists = df_selected.filter(col("artists").isin(top_artist_names)) \
    .groupBy("artists") \
    .agg(avg("popularity").alias("avg_popularity")) \
    .orderBy("avg_popularity", ascending=False)
avg_popularity_top_artists.show(truncate=False)


+------------------------------------------+-------------------+
|artists                                   |avg_popularity     |
+------------------------------------------+-------------------+
|['Queen']                                 |37.912483912483914 |
|['Die drei ???']                          |36.489885892116185 |
|['Bibi und Tina']                         |34.55987055016181  |
|['Bibi Blocksberg']                       |34.03260869565217  |
|['The Rolling Stones']                    |32.08715596330275  |
|['Elvis Presley']                         |32.02794117647059  |
|['TKKG Retro-Archiv']                     |29.189431704885344 |
|['Julio Iglesias']                        |28.46890756302521  |
|['Benjamin Blümchen']                     |28.19095143047239  |
|['Fünf Freunde']                          |27.32758620689655  |
|['Frank Sinatra']                         |25.223529411764705 |
|['Tintin', 'Tomas Bolme', 'Bert-Åke Varg']|21.716022099447514 |
|['Billie Holiday']      



While some artists appear frequently in the dataset, their average popularity may vary. This analysis helps understand whether quantity of tracks corresponds to popularity, or if a few highly popular artists dominate.


In [24]:
filtered_tracks = df_selected.filter(
    (col("year") >= 2000) &
    (col("popularity") >= 80) &
    (col("danceability") > 0.7)
)
print("Number of tracks matching criteria:", filtered_tracks.count())
filtered_tracks.select("name", "artists", "year", "popularity", "danceability").show(5, truncate=False)


Number of tracks matching criteria: 417
+----------------------+------------------+----+----------+------------+
|name                  |artists           |year|popularity|danceability|
+----------------------+------------------+----+----------+------------+
|The Real Slim Shady   |['Eminem']        |2000|84.0      |0.949       |
|Ms. Jackson           |['Outkast']       |2000|82.0      |0.843       |
|Oops!...I Did It Again|['Britney Spears']|2000|81.0      |0.751       |
|Stan                  |['Eminem', 'Dido']|2000|80.0      |0.78        |
|Without Me            |['Eminem']        |2002|85.0      |0.908       |
+----------------------+------------------+----+----------+------------+
only showing top 5 rows



In [25]:
output_path_csv = "/content/yearly_trends.csv"
yearly_trends.coalesce(1).write.mode("overwrite").option("header", True).csv(output_path_csv)
print(f"Yearly trends saved as CSV at {output_path_csv}")



Yearly trends saved as CSV at /content/yearly_trends.csv


In [26]:
output_path_parquet = "/content/yearly_trends.parquet"
yearly_trends.write.mode("overwrite").parquet(output_path_parquet)
print(f"Yearly trends saved as Parquet at {output_path_parquet}")


Yearly trends saved as Parquet at /content/yearly_trends.parquet


The yearly trends DataFrame was saved in both CSV and Parquet formats in the Colab environment.  
CSV format is human-readable and easy to use with many tools, while Parquet is a compressed, columnar storage format optimized for big data processin

In [31]:
import pyspark
import sys
print("Spark version:", pyspark.__version__)
print("Python version:", sys.version)


Spark version: 3.3.0
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
